In [6]:
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np
from matplotlib import pyplot as plt


class DecisionTrees:

    def __init__(self, split_condition=None, true_branch=None, false_branch=None, results={}, result=None, error=0):
        self.split_condition = split_condition
        self.true_branch = true_branch
        self.false_branch = false_branch
        self.results = results
        self.result = result
        self.error = error

    @classmethod
    def set_target_mapping(cls, mapping_dict, train_data):
        cls.mapping_dict = mapping_dict
        cls.train_data = train_data

    @classmethod
    def class_counts(cls, rows):
        counts = dict()
        for row in rows:
            label = row[-1]
            if cls.mapping_dict[label] not in counts:
                counts[cls.mapping_dict[label]] = 0
            counts[cls.mapping_dict[label]] += 1
        return counts

    @classmethod
    def gini_index(cls, rows):
        gini_index = 1
        counts = cls.class_counts(rows)

        for feature in counts:
            probability = counts[feature] / float(len(rows))
            gini_index -= probability ** 2
        return gini_index

    @classmethod
    def information_gain(cls, left, right, parent_node_impurity):
        p = float(len(left)) / (len(left) + len(right))
        return parent_node_impurity - p * cls.gini_index(left) - (1 - p) * cls.gini_index(right)

    @classmethod
    def partition(cls, rows, col, val):
        true_rows, false_rows = [], []
        for row in rows:
            if row[col] >= val:
                true_rows.append(row)
            else:
                false_rows.append(row)
        return true_rows, false_rows

    @classmethod
    def split_data(cls, rows):
        best_gain = 0
        best_split = None
        current_gini_index = cls.gini_index(rows)
        n_features = len(rows[0]) - 1

        for col in range(n_features):
            values = set([row[col] for row in rows])
            for val in values:
                true_rows, false_rows = cls.partition(rows, col, val)
                if len(true_rows) == 0 or len(false_rows) == 0:
                    continue
                gain = cls.information_gain(true_rows, false_rows, current_gini_index)
                if gain >= best_gain:
                    best_gain, best_split = gain, (col, val)
        return best_gain, best_split

    @classmethod
    def build_tree(cls, rows):
        results = cls.class_counts(rows)
        result = sorted(results.items(), key=lambda x: x[1], reverse=True)[0][0]
        error = 0
        for k, v in results.items():
            if k != result:
                error += v
        gain, split_condition = cls.split_data(rows)

        if gain == 0:
            return Leaf(rows, error)

        true_rows, false_rows = cls.partition(rows, split_condition[0], split_condition[1])
        true_branch = cls.build_tree(true_rows)
        false_branch = cls.build_tree(false_rows)

        return DecisionTrees(split_condition=split_condition, true_branch=true_branch, false_branch=false_branch, results=results,
                             result=result,
                             error=error)

    def print_tree(self, node, column, spacing="\t\t"):
        if isinstance(node, Leaf):
            print(f" {spacing} Predict {node.predictions}")
            return

        if node is not None:
            print(f"{spacing} is {column[node.split_condition[0]]} >= {node.split_condition[1]}")

            if node.true_branch is not None:
                print(f"{spacing} --> True:")
                self.print_tree(node.true_branch, column, spacing + "  ")

            if node.true_branch is not None:
                print(f"{spacing} --> False:")
                self.print_tree(node.false_branch, column, spacing + "  ")

    @classmethod
    def accuracy_metric(cls, actual, predicted):
        correct = 0
        for i in range(len(actual)):
            if cls.mapping_dict[actual[i]] == predicted[i]:
                correct += 1
        return correct / float(len(actual)) * 100.0

    @classmethod
    def evaluate_algorithm(cls, test_data):
        scores = list()
        test_set = list()
        for row in test_data:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = cls.decision_tree(cls.train_data, test_set)
        actual = [row[-1] for row in test_data]
        predicted = [k for d in predicted for k in d]
        accuracy = cls.accuracy_metric(actual, predicted)
        scores.append(accuracy)
        return scores

    @classmethod
    def predict(cls, node, row):
        if isinstance(node, Leaf):
            return node.predictions

        if row[node.split_condition[0]] >= node.split_condition[1]:
            return cls.predict(node.true_branch, row)
        else:
            return cls.predict(node.false_branch, row)

    @classmethod
    def decision_tree(cls, train, test):
        tree = cls.build_tree(train)
        predictions = list()
        for row in test:
            prediction = cls.predict(tree, row)
            predictions.append(prediction)
        return predictions

    @classmethod
    def minimum_error_pruning(cls, node, num_classes):
        if isinstance(node, Leaf):
            sum_ = list(node.predictions.values())[0]
            error_leaf = (node.error + num_classes - 1) / (sum_ + num_classes)
            return sum_, error_leaf

        sum_ = sum(node.results.values())
        error_leaf = (node.error + num_classes - 1) / (sum_ + num_classes)

        sum_true, error_true = cls.minimum_error_pruning(node.true_branch, num_classes)
        sum_false, error_false = cls.minimum_error_pruning(node.false_branch, num_classes)
        error_subtree = sum_true / sum_ * error_true + sum_false / sum_ * error_false

        if error_leaf <= error_subtree:
            node.true_branch = None
            node.false_branch = None
            return sum_, error_leaf
        return sum_, error_subtree

    @classmethod
    def plot_boundary(cls, X, y, node):
        n_classes = 2
        plot_colors = "ryb"
        plot_step = 0.02
        X = X.to_numpy()
        y = y.to_numpy()
        x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
        y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
        xx, yy = np.meshgrid(np.arange(x_min, x_max, plot_step), np.arange(y_min, y_max, plot_step))
        p = np.c_[xx.ravel(), yy.ravel()]

        z = []
        for row in p:
            k = cls.predict(node, row)
            z.append(int(list(k.keys())[0]))
        z = np.array(z)
        z = z.reshape(xx.shape)
        cs = plt.contourf(xx, yy, z, cmap=plt.cm.RdYlBu)

        plt.xlabel("X1")
        plt.ylabel("X2")
        a = ['X1', 'X2']
        for i, color in zip(range(n_classes), plot_colors):
            idx = np.where(y == i)
            plt.scatter(X[idx, 0], X[idx, 1], label=a[i], cmap=plt.cm.RdYlBu, edgecolor='black', s=15)

        plt.suptitle("Decision boundary for synthetic dataset")
        plt.show()


class Leaf:
    def __init__(self, rows, error=None, cnt=0):
        self.predictions = DecisionTrees.class_counts(rows)
        self.cnt = cnt
        self.error = error


def main():
    option = int(input(f"""Input the dataset you want to predict the values for from the list:
            1) IRIS dataset
            2) Synthetic dataset
            """))
    if option == 1:
        data, target = datasets.load_iris(return_X_y=True, as_frame=True)
        mapping_dict = {0: 'setosa', 1: 'versicolor', 2: 'virginica'}
        num_classes = target.nunique()

    elif option == 2:
        df = pd.read_csv("dt_data.csv")
        data = df[["X1", "X2"]]
        target = df[["Y"]]
        mapping_dict = {0: 0, 1: 1}
        num_classes = target.nunique().item()

    else:
        print("Please select a valid dataset from the list")
    column = data.columns
    dataset = pd.concat([data, target], axis=1)
    l = dataset.values.tolist()

    train_data, test_data = train_test_split(l, random_state=5)

    DecisionTrees.set_target_mapping(mapping_dict, train_data)
    reg = DecisionTrees()
    my_tree = reg.build_tree(train_data)

    # Print and output score for full tree
    reg.print_tree(my_tree, column)
    accuracy = reg.evaluate_algorithm(test_data)
    print(f"Full tree accuracy: {accuracy} ")

    reg.minimum_error_pruning(my_tree, num_classes)

    # Print and output score for Pruned tree
    reg.print_tree(my_tree, column)
    accuracy = reg.evaluate_algorithm(test_data)
    print(f"Pruned tree accuracy: {accuracy} ")

    # Decision Boundary plotting for synthetic dataset
    if option == 2:
        dt = reg.build_tree(l)
        reg.plot_boundary(data, target, dt)


if __name__ == '__main__':
    main()


Input the dataset you want to predict the values for from the list:
            1) IRIS dataset
            2) Synthetic dataset
            1
		 is petal width (cm) >= 1.0
		 --> True:
		   is petal width (cm) >= 1.8
		   --> True:
 		     Predict {'virginica': 35}
		   --> False:
		     is petal length (cm) >= 5.8
		     --> True:
 		       Predict {'virginica': 1}
		     --> False:
		       is sepal length (cm) >= 5.0
		       --> True:
		         is petal length (cm) >= 5.0
		         --> True:
		           is petal width (cm) >= 1.6
		           --> True:
 		             Predict {'versicolor': 2}
		           --> False:
 		             Predict {'virginica': 1}
		         --> False:
 		           Predict {'versicolor': 34}
		       --> False:
 		         Predict {'virginica': 1}
		 --> False:
 		   Predict {'setosa': 38}
Full tree accuracy: [94.73684210526315] 
		 is petal width (cm) >= 1.0
		 --> True:
		   is petal width (cm) >= 1.8
		   --> True:
 		     Predict {'virginica': 35